# Gauging Sentiment in Yelp Reviews
### CONVERT JSON TO CSV

YELP DATASET DOCUMENTATION:  https://www.yelp.com/dataset/documentation/main

In [2]:
!pip install simplejson

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [33]:
import argparse
import collections
import csv
import simplejson as json

In [34]:
def read_and_write_file(json_file_path, csv_file_path, column_names):
    """Read in the json dataset file and write it out to a csv file, given the column names."""
    with open(csv_file_path, 'w',encoding="utf-8") as fout:
        csv_file = csv.writer(fout)
        csv_file.writerow(list(column_names))
        with open(json_file_path, encoding="utf-8") as fin:
            for line in fin:
                line_contents = json.loads(line)
                #print(column_names, line_contents)
                csv_file.writerow(get_row(line_contents, column_names))


In [35]:
def get_superset_of_column_names_from_file(json_file_path):
    """Read in the json dataset file and return the superset of column names."""
    column_names = set()
    with open(json_file_path,encoding="utf-8") as fin:
        for line in fin:
            line_contents = json.loads(line)
            column_names.update(
                    set(get_column_names(line_contents).keys())
                    )
    return column_names

In [36]:
def get_column_names(line_contents, parent_key=''):
    """Return a list of flattened key names given a dict.

    Example:

        line_contents = {
            'a': {
                'b': 2,
                'c': 3,
                },
        }

        will return: ['a.b', 'a.c']

    These will be the column names for the eventual csv file.

    """
    column_names = []
    for k, v in line_contents.items():
        column_name = "{0}.{1}".format(parent_key, k) if parent_key else k
        if isinstance(v, collections.MutableMapping):
            column_names.extend(
                    get_column_names(v, column_name).items()
                    )
        else:
            column_names.append((column_name, v))
    return dict(column_names)

In [42]:
def get_nested_value(d, key):
    """Return a dictionary item given a dictionary `d` and a flattened key from `get_column_names`.
    
    Example:

        d = {
            'a': {
                'b': 2,
                'c': 3,
                },
        }
        key = 'a.b'

        will return: 2
    
    """
    if '.' not in key:
        if key not in d:
            return None
        return d[key]
    base_key, sub_key = key.split('.', 1)
    if base_key not in d:
        return None
    sub_dict = d[base_key]
    if sub_dict is None:
        return None
    
    
    return get_nested_value(sub_dict, sub_key)

In [38]:
def get_row(line_contents, column_names):
    """Return a csv compatible row given column names and a dict."""
    row = []
    for column_name in column_names:
        line_value = get_nested_value(line_contents, column_name,)
        # print (line_value)
        if isinstance(line_value, str):
            row.append(line_value)
        elif line_value is not None:
            row.append(line_value)
        else:
            row.append('')
    #print(row)
    return row

In [39]:
json_file = 'user.json'
csv_file = '{0}.csv'.format(json_file.split('.json')[0])

column_names = get_superset_of_column_names_from_file(json_file)
read_and_write_file(json_file, csv_file, column_names)

In [43]:
json_file = 'review.json'
csv_file = '{0}.csv'.format(json_file.split('.json')[0])

column_names = get_superset_of_column_names_from_file(json_file)
read_and_write_file(json_file, csv_file, column_names)

In [73]:
json_file = 'business.json'
csv_file = '{0}.csv'.format(json_file.split('.json')[0])

# skip attributes and hours attributes
column_names = ['business_id','name','address','city','state','postal code','latitude', 'longitude', 'stars', 'review_count', 'is_open', 'categories']
read_and_write_file(json_file, csv_file, column_names)

In [2]:
import pandas as pd
import numpy as np

### STORE CSV IN PANDAS DATAFRAME

In [45]:
users = pd.read_csv('user.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [56]:
# reorder columns to match documentation
users = users[['user_id','name','review_count','yelping_since', 'friends', 'useful', 'funny', 'cool', 'fans', 'elite', 'average_stars', 'compliment_hot', 'compliment_more', 'compliment_profile', 'compliment_cute', 'compliment_list', 'compliment_note', 'compliment_plain', 'compliment_cool', 'compliment_funny', 'compliment_writer', 'compliment_photos' ]]

In [55]:
users.head()

,user_id,name,review_count,yelping_since,friends,useful,funny,cool,fans,elite,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,l6BmjZMeQD3rDxWUbiAiow,Rashmi,95,2013-10-08 23:11:33,"c78V-rj8NQcQjOI8KP3UEA, alRMgPcngYSCJ5naFRBz5g...",84,17,25,5,"2015,2016,2017",...,0,0,0,0,1,1,1,1,2,0
1,4XChL029mKr5hydo79Ljxg,Jenna,33,2013-02-21 22:29:06,"kEBTgDvFX754S68FllfCaA, aB2DynOxNOJK9st2ZeGTPg...",48,22,16,4,NaN,...,0,0,0,0,0,0,1,1,0,0
2,bc8C_eETBWL0olvFSJJd0w,David,16,2013-10-04 00:16:10,"4N-HU_T32hLENLntsNKNBg, pSY2vwWLgWfGVAAiKQzMng...",28,8,10,0,NaN,...,0,0,0,0,1,0,0,0,0,0
3,dD0gZpBctWGdWo9WlGuhlA,Angela,17,2014-05-22 15:57:30,"RZ6wS38wnlXyj-OOdTzBxA, l5jxZh1KsgI8rMunm-GN6A...",30,4,14,5,NaN,...,0,0,0,0,0,2,0,0,1,0
4,MM4RJAeH6yuaN8oZDSt0RA,Nancy,361,2013-10-23 07:02:50,"mbwrZ-RS76V1HoJ0bF_Geg, g64lOV39xSLRZO0aQQ6DeQ...",1114,279,665,39,"2015,2016,2017,2018",...,1,0,0,1,16,57,80,80,25,5


In [3]:
reviews = pd.read_csv('review.csv')

In [4]:
# reorder columns to match documentation
reviews = reviews[['review_id', 'user_id', 'business_id', 'stars', 'date', 'text', 'useful', 'funny', 'cool']]

In [5]:
reviews.head(40)

,review_id,user_id,business_id,stars,date,text,useful,funny,cool
0,Q1sbwvVQXV2734tPgoKj4Q,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,1.0,2013-05-07 04:34:36,Total bill for this horrible service? Over $8G...,6,1,0
1,GJXCdrto3ASJOqKeVWPi6Q,yXQM5uF2jS6es16SJzNHfg,NZnhc2sEQy3RmzKTZnqtwQ,5.0,2017-01-14 21:30:33,I *adore* Travis at the Hard Rock's new Kelly ...,0,0,0
2,2TzJjDVDEuAW6MR5Vuc1ug,n6-Gk65cPZL6Uz8qRm3NYw,WTqjgwHlXbSFevF32_DJVw,5.0,2016-11-09 20:09:03,I have to say that this office really has it t...,3,0,0
3,yi0R0Ugj_xUx_Nek0-_Qig,dacAIZ6fTM6mqwW5uxkskg,ikCg8xy5JIg_NGPx-MSIDA,5.0,2018-01-09 20:56:38,Went in for a lunch. Steak sandwich was delici...,0,0,0
4,11a8sVPMUFtaC7_ABRkmtw,ssoyf2_x0EQMed6fgHeMyQ,b1b1eb3uo-w561D0ZfCEiQ,1.0,2018-01-30 23:07:38,Today was my second out of three sessions I ha...,7,0,0
5,fdiNeiN_hoCxCMy2wTRW9g,w31MKYsNFMrjhWxxAb5wIw,eU_713ec6fTGNO4BegRaww,4.0,2013-01-20 13:25:59,I'll be the first to admit that I was not exci...,0,0,0
6,G7XHMxG0bx9oBJNECG4IFg,jlu4CztcSxrKx56ba1a5AQ,3fw2X5bZYeW9xCz_zGhOHg,3.0,2016-05-07 01:21:02,Tracy dessert had a big name in Hong Kong and ...,5,4,5
7,8e9HxxLjjqc9ez5ezzN7iQ,d6xvYpyzcfbF_AZ8vMB7QA,zvO-PJCpNk4fgAVUnExYAA,1.0,2010-10-05 19:12:35,This place has gone down hill. Clearly they h...,3,1,1
8,qrffudO73zsslZbe8B9D3Q,sG_h0dIzTKWa3Q6fmb4u-g,b2jN2mm9Wf3RcrZCgfo1cg,2.0,2015-01-18 14:04:18,I was really looking forward to visiting after...,1,0,0
9,RS_GTIT6836bCaPy637kNQ,nMeCE5-xsdleyxYuNZ_7rA,oxwGyA17NL6c5t1Etg5WgQ,3.0,2012-02-29 21:52:43,It's a giant Best Buy with 66 registers. I do...,1,0,1


In [74]:
businesses = pd.read_csv('business.csv')

In [75]:
businesses.head()

,business_id,name,address,city,state,postal code,latitude,longitude,stars,review_count,is_open,categories
0,1SWheh84yJXfytovILXOAQ,Arizona Biltmore Golf Club,2818 E Camino Acequia Drive,Phoenix,AZ,NaN,33.522143,-112.018481,3.0,5,0,"Golf, Active Life"
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,NaN,43.605499,-79.652289,2.5,128,1,"Specialty Food, Restaurants, Dim Sum, Imported..."
2,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,"10110 Johnston Rd, Ste 15",Charlotte,NC,NaN,35.092564,-80.859132,4.0,170,1,"Sushi Bars, Restaurants, Japanese"
3,xvX2CttrVhyG2z1dFg_0xw,Farmers Insurance - Paul Lorenz,"15655 W Roosevelt St, Ste 237",Goodyear,AZ,NaN,33.455613,-112.395596,5.0,3,1,"Insurance, Financial Services"
4,HhyxOkGAM07SRYtlQ4wMFQ,Queen City Plumbing,"4209 Stuart Andrew Blvd, Ste F",Charlotte,NC,NaN,35.190012,-80.887223,4.0,4,1,"Plumbing, Shopping, Local Services, Home Servi..."


In [99]:
# reviews with ratings of 1 star

stars_1 = reviews[(reviews.stars == 1.0)]
stars_1.head()

file_name = 'stars1.csv'
stars_1.to_csv(file_name, sep=',', encoding='utf-8')

In [103]:
# reviews with ratings of 2 stars

stars_2 = reviews[(reviews.stars == 2.0)]
stars_2.head()

file_name = 'stars2.csv'
stars_2.to_csv(file_name, sep=',', encoding='utf-8')

In [104]:
# reviews with ratings of 3 stars

stars_3 = reviews[(reviews.stars == 3.0)]
stars_3.head()

file_name = 'stars3.csv'
stars_3.to_csv(file_name, sep=',', encoding='utf-8')

In [105]:
# reviews with ratings of 4 stars

stars_4 = reviews[(reviews.stars == 4.0)]
stars_4.head()

file_name = 'stars4.csv'
stars_4.to_csv(file_name, sep=',', encoding='utf-8')

In [106]:
# reviews with ratings of 4 stars

stars_5 = reviews[(reviews.stars == 5.0)]
stars_5.head()

file_name = 'stars5.csv'
stars_5.to_csv(file_name, sep=',', encoding='utf-8')

In [110]:
# generate small sample for portability

reviews_subset = reviews.sample(n = 5000) 
reviews_subset.head()

,review_id,user_id,business_id,stars,date,text,useful,funny,cool
1162113,tgMs2QSZ3aJOVNufv1cTnQ,pWfESevdUDX85UlIRn1_hA,z9KxcVoe6tQGyB_Zy0Umcg,5.0,2017-06-02 22:45:41,We got a restaurant.com certificate to try thi...,0,0,0
4112947,fTLQmHC4OGcpsruIhAVuVA,0hZfE2He1YssM_wEvKhfcA,4VPjBVJKcxDpr00OMwxSww,5.0,2016-09-07 05:04:17,Just the smells walking up to this place get t...,1,0,0
5086852,Np7tRTyKV0oUqUp9sptP6A,5VHWTXocEYwxoJe0vG1BJA,1PpmlyzOiL_DYh0Ty4VC6A,4.0,2017-02-15 18:39:38,"This is quite a late review, but better late t...",0,0,0
3809621,QMYiq_1bLMIfphPmTotDNA,iSC96O2NjQc3JExGUHQG0Q,Bm8nRUsZ-dK6g2eJLxMTOw,4.0,2013-12-02 05:43:38,I came too Naka Sushi after solid 16 mile bike...,6,1,5
3728083,cBWuetrpzkE38NVDr4W3ag,5hCUYYq6Go3-ZTpSeVVMiA,BYQXVyuarIvLi32JzMLSVg,1.0,2016-07-13 02:16:41,"Owner owes $3,000 for yogurt mix and refuses t...",2,2,1


In [111]:
file_name = 'reviews5000.csv'
reviews_subset.to_csv(file_name, sep=',', encoding='utf-8')

In [113]:
reviews_subset2 = reviews.sample(n = 10000) 
reviews_subset2.head()

,review_id,user_id,business_id,stars,date,text,useful,funny,cool
4694397,Llol4TRuLoVmCsbVSnuIDA,75JyTel9uUs7VNFhTmK56w,sPwgafejYVdf8hd4-oDYVg,1.0,2015-08-30 17:32:01,Awful. The lemon pepper wing sauce is black fo...,0,0,0
3142277,moHHEwAINiwsjohq6doYrQ,lhDgwKq2Gh8TIIyexubRag,9U4Tqt5SkZKQ6eeH6IPSxQ,5.0,2018-04-11 04:36:23,We had an awesome experience at hibachi. Our c...,0,0,1
2342009,6TF21--YL6ig6r0Oae1qGA,jpib9MAMB_aVxyCDUi9VAg,yFUvKpW66cNsYXD6inDsog,5.0,2015-01-27 03:54:16,I truly enjoy this neighborhood spot. Good foo...,0,0,0
3547267,MN2PqoIWECNmI3fuqdZyEA,bFF1HZzb8k3gimLHU52bQA,WnOVGJTu7RK0iWurrgiBNA,5.0,2018-06-28 19:06:47,Best tanning place around in NC\r\nHope and he...,0,0,0
4449537,MG4o8ke2G6VYXDy1HUCbIg,PT3whiZ1mW3psCz_lcp7ww,6BO1PcdtU6idqO0njUnCYA,1.0,2016-07-12 16:49:09,Staff is horrible. Last time I ordered from he...,0,0,1


In [114]:
file_name = 'reviews10000.csv'
reviews_subset2.to_csv(file_name, sep=',', encoding='utf-8')

In [ ]:
reviews_subset2 = reviews.sample(n = 25000) 
reviews_subset2.head()